In [1]:
import json
import numpy as np
from model_balancing import ModelBalancing, Q_
import cvxpy as cp
np.set_printoptions(precision=1)

In [2]:
with open("../cvxpy/examples/simple-example/no_noise_with_noisefree_kinetic_data/small_model_network_and_data.json", "rt") as fp:
    data = json.load(fp)

args = {}
args["S"] = np.array(data["network"]["stoichiometric_matrix"])
args["A_act"] = np.array(data["network"]["activation_matrix"])
args["A_inh"] = np.array(data["network"]["inhibition_matrix"])
args["fluxes"] = Q_(data["reaction_fluxes"]["data"]["mean"], data["reaction_fluxes"]["unit"])

args["Keq_gmean"] = Q_(data["kinetic_constants"]["Keq"]["combined"]["geom_mean"], "")
args["Keq_gmean"][2] *= 1000  # fixing standard concentration convension (from 1 mM to 1 M)
args["Keq_ln_cov"] = np.array(data["kinetic_constants"]["Keq"]["combined"]["cov_ln"])

args["kcatf_gmean"] = Q_(data["kinetic_constants"]["Kcatf"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatf"]["unit"])
args["kcatf_ln_cov"] = np.array(data["kinetic_constants"]["Kcatf"]["combined"]["cov_ln"])

args["kcatr_gmean"] = Q_(data["kinetic_constants"]["Kcatr"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatr"]["unit"])
args["kcatr_ln_cov"] = np.array(data["kinetic_constants"]["Kcatr"]["combined"]["cov_ln"])

args["Km_gmean"] = Q_(data["kinetic_constants"]["KM"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KM"]["unit"])
args["Km_ln_cov"] = np.array(data["kinetic_constants"]["KM"]["combined"]["cov_ln"])

args["Ka_gmean"] = Q_(data["kinetic_constants"]["KA"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KA"]["unit"])
args["Ka_ln_cov"] = np.array(data["kinetic_constants"]["KA"]["combined"]["cov_ln"])

args["Ki_gmean"] = Q_(data["kinetic_constants"]["KI"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KI"]["unit"])
args["Ki_ln_cov"] = np.array(data["kinetic_constants"]["KI"]["combined"]["cov_ln"])

args["conc_met_gmean"] = Q_(data["metabolite_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_met_gstd"] = np.array(data["metabolite_concentrations"]["combined"]["geom_std"])

args["conc_enz_gmean"] = Q_(data["enzyme_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_enz_gstd"] = np.array(data["enzyme_concentrations"]["combined"]["geom_std"])

args["rate_law"] = "CM"

In [3]:
mb = ModelBalancing(**args)
mb.initialize_with_gmeans()
print("kcat reverse comparison:")
print("from JSON:  ", args["kcatr_gmean"].m_as("1/s"))
print("calculated: " , np.exp(mb.ln_kcatr.value))

kcat reverse comparison:
from JSON:   [ 814.9 1149.9  199.8   41.8 1722.8]
calculated:  [3.4e+03 3.5e-02 2.7e+02 6.8e+01 2.4e+02]


In [4]:
print("enzyme concentration comparison:")
print("from JSON:  \n", args["conc_enz_gmean"].m_as("M"))
print("calculated: \n" , np.exp(mb.ln_conc_enz.value))

enzyme concentration comparison:
from JSON:  
 [[4.6e-07 1.1e-06 1.1e-06 6.9e-07]
 [3.0e-06 1.4e-06 1.2e-06 7.6e-07]
 [1.2e-06 7.0e-07 5.0e-07 6.8e-07]
 [5.8e-07 5.9e-07 2.4e-07 6.1e-07]
 [9.2e-07 7.2e-07 4.1e-07 4.0e-07]]
calculated: 
 [[3.9e-07 4.4e-07 6.2e-07 3.7e-07]
 [4.8e-04 3.3e-04 3.3e-04 3.1e-04]
 [1.1e-06 9.1e-07 5.9e-07 6.1e-07]
 [4.3e-07 7.6e-07 3.2e-07 5.8e-07]
 [3.8e-07 5.5e-07 4.7e-07 3.9e-07]]


In [5]:
%%time

mb = ModelBalancing(**args)
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

optimized total squared Z-scores = 154.765
------------------------- All Z-scores -------------------------
enz = 0.37
met = 2.9
Keq = 6.4
kcatf = 40.58
kcatr = 32.37
Km = 72.15
Ka = 0.0
Ki = 0.0
CPU times: user 18.6 s, sys: 2.74 s, total: 21.4 s
Wall time: 20.2 s


In [6]:
%%time

mb = ModelBalancing(**args)

print("Setting enzyme conc. and kcatr gmeans to calculated values...")
# check that the driving forces at the mode are all positive
assert mb.is_gmean_feasible()

mb.initialize_with_gmeans()
print(f"initial total squared Z-scores = {mb.objective_value:.3f}")
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

Setting enzyme conc. and kcatr gmeans to calculated values...
initial total squared Z-scores = 0.000
optimized total squared Z-scores = 0.000
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.0
Keq = 0.0
kcatf = 0.0
kcatr = 0.0
Km = 0.0
Ka = 0.0
Ki = 0.0
CPU times: user 18.8 s, sys: 2.13 s, total: 20.9 s
Wall time: 20.3 s
